In [145]:
import numpy as np
import pandas as pd
import pysal as ps
import os
os.chdir('/Users/toshan/dev/pysal/pysal/weights')
from spintW import ODW
from pysal.spreg.twosls_sp import GM_Lag, BaseGM_Lag

In [146]:
#Prep Data
austria = pd.read_csv('http://dl.dropbox.com/u/8649795/AT_Austria.csv')
f = austria['Data'].values.reshape((-1,1))
o = austria['Origin'].values.reshape((-1,1))
d = austria['Destination'].values.reshape((-1,1))
dij = austria['Dij'].values.reshape((-1,1))
o_vars = austria['Oi2007'].values.reshape((-1,1))
d_vars = austria['Dj2007'].values.reshape((-1,1))

Wo = ps.weights.lat2W(3,3)
Wd = ps.weights.lat2W(3,3)
Ww = ODW(Wo,Wd)

y = f.reshape((-1,1))
X = np.hstack([o_vars.reshape((-1,1)), d_vars.reshape((-1,1)), dij.reshape((-1,1))])

In [149]:
#GM Lag with OD-W
model = GM_Lag(y, X, w=ODw, w_lags=1)
model.betas

array([[ -2.86692386e+02],
       [  1.02229167e-01],
       [  1.02020709e-01],
       [ -3.59211608e+00],
       [ -3.46757679e-02]])

In [150]:
#Base TSLS with OD-W

X = np.hstack([np.ones((81,1)), o_vars.reshape((-1,1)), d_vars.reshape((-1,1)), dij.reshape((-1,1))])
#X = np.hstack([o_vars.reshape((-1,1)), d_vars.reshape((-1,1)), dij.reshape((-1,1))])

Wo = ps.weights.full2W(np.kron(Wo.full()[0], np.identity(9)))
Wd = ps.weights.full2W(np.kron(np.identity(9), Wd.full()[0]))

w_lags=1
    
o_yend, o_q = set_endog(y, X, Wo, None, None, w_lags, True)
d_yend, d_q = set_endog(y, X, Wd, None, None, w_lags, True)
w_yend, w_q = set_endog(y, X, Ww, None, None, w_lags, True)

w = Ww.sparse
yend = w_yend
q = w_q

model = TSLS.BaseTSLS(y=y, x=X, yend=yend, q=q)
model.betas

array([[  1.40576692e+03],
       [  6.93836657e-02],
       [  6.03194419e-02],
       [ -1.05386559e+01],
       [  3.10794995e-01]])

In [151]:
#Base TSLS with O-W, D-Wm and OD-W

w = (Wo.sparse, Wd.sparse, Ww.sparse)
yend = np.hstack([o_yend, d_yend, w_yend])
q = np.hstack([o_q, d_q, w_q])

model = TSLS.BaseTSLS(y=f, x=X, yend=yend, q=q)
model.betas

array([[  5.38733321e+04],
       [ -1.33117926e+01],
       [ -7.14188368e+00],
       [  3.55723696e+02],
       [  7.88125684e+00],
       [  1.20094221e+01],
       [  6.54489799e+01]])